In [28]:
import pandas as pd

api_url = "https://api.nhtsa.gov/SafetyRatings/VehicleId"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3",
}
min_vehicle_id = 25000
max_vehicle_id = 35000

In [13]:
import requests


def fetch_vehicle_data(vehicle_id: int) -> dict:
    try:
        response = requests.get(f"{api_url}/{vehicle_id}", headers=headers)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Failed to retrieve data for vehicle ID {vehicle_id}: {e}")
        return {}

In [14]:
result = fetch_vehicle_data(1)

print(result)

{'Count': 1, 'Message': 'Results returned successfully', 'Results': [{'VehiclePicture': 'https://static.nhtsa.gov/images/vehicles/7284_st0640_046.png', 'OverallRating': 'Not Rated', 'OverallFrontCrashRating': 'Not Rated', 'FrontCrashDriversideRating': 'Not Rated', 'FrontCrashPassengersideRating': 'Not Rated', 'OverallSideCrashRating': 'Not Rated', 'SideCrashDriversideRating': 'Not Rated', 'SideCrashPassengersideRating': 'Not Rated', 'combinedSideBarrierAndPoleRating-Front': 'Not Rated', 'combinedSideBarrierAndPoleRating-Rear': 'Not Rated', 'sideBarrierRating-Overall': 'Not Rated', 'RolloverRating': 'Not Rated', 'RolloverRating2': 'Not Rated', 'RolloverPossibility': 0.0, 'RolloverPossibility2': 0.0, 'dynamicTipResult': ' ', 'SidePoleCrashRating': 'Not Rated', 'NHTSAElectronicStabilityControl': 'Standard', 'NHTSAForwardCollisionWarning': 'No', 'NHTSALaneDepartureWarning': 'No', 'ComplaintsCount': 42, 'RecallsCount': 4, 'InvestigationCount': 2, 'ModelYear': 2011, 'Make': 'AUDI', 'Model': 

We need to program a function to each hour retrieve all database from id 1 to id 25000 in parallel

In [22]:
import concurrent.futures
from tqdm import tqdm


def fetch_all_vehicle_data_in_parallel(min_id: int, max_id: int) -> dict[list]:
    # with concurrent.futures.ThreadPoolExecutor() as executor:
    results = []
    for i in tqdm(range(min_id, max_id + 1)):
        cdict = fetch_vehicle_data(i)
        if cdict != {}:
            results.append(cdict)
        else:
            time.sleep(10)
            print("Waiting 10 seconds, as we were forbidded at index" + str(i))
    return {"apiResponse": results}

In [16]:
output_folder = "data/landing/temporal"

In [17]:
import os
import json


def save_vehicle_data(data, filepath):
    with open(filepath, "w") as f:
        json.dump(data, f, indent=4)

In [29]:
min_vehicle_id, max_vehicle_id

(2500, 25000)

In [30]:
all_vehicle_data = fetch_all_vehicle_data_in_parallel(min_vehicle_id, max_vehicle_id)

100%|██████████| 22501/22501 [2:19:38<00:00,  2.69it/s]  


In [21]:
print(all_vehicle_data)

{'apiResponse': [{'Count': 1, 'Message': 'Results returned successfully', 'Results': [{'VehiclePicture': 'https://static.nhtsa.gov/images/vehicles/7284_st0640_046.png', 'OverallRating': 'Not Rated', 'OverallFrontCrashRating': 'Not Rated', 'FrontCrashDriversideRating': 'Not Rated', 'FrontCrashPassengersideRating': 'Not Rated', 'OverallSideCrashRating': 'Not Rated', 'SideCrashDriversideRating': 'Not Rated', 'SideCrashPassengersideRating': 'Not Rated', 'combinedSideBarrierAndPoleRating-Front': 'Not Rated', 'combinedSideBarrierAndPoleRating-Rear': 'Not Rated', 'sideBarrierRating-Overall': 'Not Rated', 'RolloverRating': 'Not Rated', 'RolloverRating2': 'Not Rated', 'RolloverPossibility': 0.0, 'RolloverPossibility2': 0.0, 'dynamicTipResult': ' ', 'SidePoleCrashRating': 'Not Rated', 'NHTSAElectronicStabilityControl': 'Standard', 'NHTSAForwardCollisionWarning': 'No', 'NHTSALaneDepartureWarning': 'No', 'ComplaintsCount': 42, 'RecallsCount': 4, 'InvestigationCount': 2, 'ModelYear': 2011, 'Make': 

In [ ]:
import pandas as pd

In [8]:
import time

timestamp = time.strftime("%Y%m%d")
filepath = f"{output_folder}/NHTSA_-_Safety_Rating_{timestamp}.json"
save_vehicle_data(all_vehicle_data, filepath)

FileNotFoundError: [Errno 2] No such file or directory: 'data/landing/temporal/NHTSA_-_Safety_Rating_20241027.json'

In [9]:
import os

root_folder = os.path.join(os.path.abspath(os.getcwd()), "..", "..")
output_folder = os.path.join(root_folder, "data", "landing", "temporal")

output_folder

'/Users/ignasi/Documents/_03_MDS_/_01_ADSDB_/notebooks/landing/../../data/landing/temporal'

In [10]:
import time

timestamp = time.strftime("%Y%m%d")
filepath = f"{output_folder}/NHTSA_-_Safety_Rating_{timestamp}.json"
save_vehicle_data(all_vehicle_data, filepath)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/ignasi/Documents/_03_MDS_/_01_ADSDB_/notebooks/landing/../../data/landing/temporal/NHTSA_-_Safety_Rating_20241027.json'

In [31]:
save_vehicle_data(all_vehicle_data, "./20241027.json")

In [26]:
timestamp

'20241027'

In [33]:
len(all_vehicle_data["apiResponse"])

22501